<a href="https://colab.research.google.com/github/buzzaggwd/NLP/blob/main/npl_lab3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# текст и его токенизация
text = "Далеко-далеко за словесными горами в стране гласных и согласных живут рыбные тексты. Вдали от всех живут они в буквенных домах на берегу Семантика большого языкового океана. Маленький ручеек Даль журчит по всей стране и обеспечивает ее всеми необходимыми правилами. Эта парадигматическая страна, в которой жаренные члены предложения залетают прямо в рот. Даже всемогущая пунктуация не имеет власти над рыбными текстами, ведущими безорфографичный образ жизни. Однажды одна маленькая строчка рыбного текста по имени Lorem ipsum решила выйти в большой мир грамматики. Великий Оксмокс предупреждал ее о злых запятых, диких знаках вопроса и коварных точках с запятой, но текст не дал сбить себя с толку. Он собрал семь своих заглавных букв, подпоясал инициал за пояс и пустился в дорогу. Взобравшись на первую вершину курсивных гор, бросил он последний взгляд назад, на силуэт своего родного города Буквоград, на заголовок деревни Алфавит и на подзаголовок своего переулка Строчка. Грустный риторический вопрос скатился по его щеке и он продолжил свой путь. По дороге встретил текст рукопись. Она предупредила его: «В моей стране все переписывается по несколько раз. Единственное, что от меня осталось, это приставка «и». Возвращайся ты лучше в свою безопасную страну». Не послушавшись рукописи, наш текст продолжил свой путь."
tokens = text.lower().replace('\n', ' ').replace('.', '').split()
vocab = sorted(set(tokens))
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

vocab_size = len(vocab)
embedding_dim = 10
context_size = 3

data = [word_to_idx[word] for word in tokens]
X_indices, y_indices = [], []
for i in range(len(data) - context_size):
    X_indices.append(data[i:i + context_size])
    y_indices.append(data[i + context_size])

X_indices = np.array(X_indices)
y_indices = np.array(y_indices)

class SimpleLanguageModel:
    def __init__(self, vocab_size, embed_dim, context_size, learning_rate=0.01):
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.context_size = context_size
        self.lr = learning_rate

        np.random.seed(42)
        self.W1 = np.random.randn(context_size * embed_dim, 128) * 0.01
        self.W2 = np.random.randn(128, vocab_size) * 0.01
        self.embeddings = np.random.randn(vocab_size, embed_dim) * 0.01

    def forward(self, X_idx):
        x_emb = self.embeddings[X_idx]
        self.x_flat = x_emb.reshape(-1, self.context_size * self.embed_dim)

        self.h = np.tanh(self.x_flat @ self.W1)
        logits = self.h @ self.W2
        return logits

    def train_step(self, X_idx, target_idx):

        logits = self.forward(X_idx)

        exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
        probs = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

        batch_size = target_idx.shape[0]
        loss = -np.mean(np.log(probs[np.arange(batch_size), target_idx] + 1e-9))

        dlogits = probs.copy()
        dlogits[np.arange(batch_size), target_idx] -= 1
        dlogits /= batch_size

        dW2 = self.h.T @ dlogits
        dh = dlogits @ self.W2.T
        dW1 = self.x_flat.T @ (dh * (1 - self.h**2))

        self.W1 -= self.lr * dW1
        self.W2 -= self.lr * dW2

        return loss

    def train(self, X, y, epochs=100, batch_size=32):
        for epoch in range(epochs):
            total_loss = 0
            indices = np.random.permutation(len(X))

            for i in range(0, len(X), batch_size):
                batch_X = X[indices[i:i+batch_size]]
                batch_y = y[indices[i:i+batch_size]]

                loss = self.train_step(batch_X, batch_y)
                total_loss += loss

            avg_loss = total_loss / len(X)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

model = SimpleLanguageModel(vocab_size, embedding_dim, context_size)
model.train(X_indices, y_indices, epochs=50)

test_idx = X_indices[0]
logits = model.forward(test_idx[np.newaxis, :])
predicted_idx = np.argmax(logits)
print("\nТест:")
print("Контекст:", [idx_to_word[i] for i in test_idx])
print("Предсказание:", idx_to_word[predicted_idx])
print("Ожидалось:", idx_to_word[y_indices[0]])

Epoch 1/50, Loss: 0.1582
Epoch 2/50, Loss: 0.1582
Epoch 3/50, Loss: 0.1582
Epoch 4/50, Loss: 0.1582
Epoch 5/50, Loss: 0.1582
Epoch 6/50, Loss: 0.1582
Epoch 7/50, Loss: 0.1582
Epoch 8/50, Loss: 0.1582
Epoch 9/50, Loss: 0.1582
Epoch 10/50, Loss: 0.1582
Epoch 11/50, Loss: 0.1582
Epoch 12/50, Loss: 0.1582
Epoch 13/50, Loss: 0.1582
Epoch 14/50, Loss: 0.1582
Epoch 15/50, Loss: 0.1582
Epoch 16/50, Loss: 0.1582
Epoch 17/50, Loss: 0.1582
Epoch 18/50, Loss: 0.1582
Epoch 19/50, Loss: 0.1582
Epoch 20/50, Loss: 0.1582
Epoch 21/50, Loss: 0.1582
Epoch 22/50, Loss: 0.1582
Epoch 23/50, Loss: 0.1582
Epoch 24/50, Loss: 0.1582
Epoch 25/50, Loss: 0.1582
Epoch 26/50, Loss: 0.1582
Epoch 27/50, Loss: 0.1582
Epoch 28/50, Loss: 0.1582
Epoch 29/50, Loss: 0.1582
Epoch 30/50, Loss: 0.1582
Epoch 31/50, Loss: 0.1582
Epoch 32/50, Loss: 0.1582
Epoch 33/50, Loss: 0.1582
Epoch 34/50, Loss: 0.1582
Epoch 35/50, Loss: 0.1582
Epoch 36/50, Loss: 0.1582
Epoch 37/50, Loss: 0.1582
Epoch 38/50, Loss: 0.1582
Epoch 39/50, Loss: 0.